# Customer.IO Multi-API Usage

## Purpose
This notebook demonstrates how to use multiple Customer.IO APIs together:
- Data Pipelines API (existing utils)
- App API (new app_utils)
- Webhook processing (new webhook_utils)

## Setup and Imports

In [ ]:
import sys# For local development with utils imports (backwards compatibility)# Add src directory to path if it existsnotebook_dir = os.path.dirname(os.path.abspath(''))project_root = os.path.abspath(os.path.join(notebook_dir, '..', '..'))src_path = os.path.join(project_root, 'src')if os.path.exists(src_path):    sys.path.insert(0, src_path)    # Try importing from utils (local development pattern)    try:        from utils.api_client import CustomerIOClient        from utils.people_manager import identify_user        from utils.event_manager import track_event    except ImportError:        # Fall back to package imports        from pipelines_api.api_client import CustomerIOClient        from pipelines_api.people_manager import identify_user        from pipelines_api.event_manager import track_eventelse:    # Databricks or package installation - use package imports    from pipelines_api.api_client import CustomerIOClient    from pipelines_api.people_manager import identify_user    from pipelines_api.event_manager import track_event# Data Pipelines API (existing)# App API (new)from app_utils.auth import AppAPIAuthfrom app_utils.client import send_transactional, trigger_broadcast, send_push# Webhook processing (new)from webhook_utils.processor import verify_signature, parse_event, get_event_typeimport osimport json

## Databricks Environment SetupThis section handles the installation of required libraries and configuration for Databricks clusters. Skip this section if running locally.

In [ ]:
# Databricks library installation# Uncomment the following lines when running on Databricks# %pip install customerio-api-client# OR if using a wheel file:# %pip install /dbfs/path/to/customerio_api_client-1.0.0-py3-none-any.whl# For development without packaging, install individual requirements:# %pip install httpx>=0.25.0 pydantic>=2.0.0 pandas>=2.0.0 python-dotenv>=1.0.0 structlog>=24.0.0# Restart kernel after installation if needed# import dbutils# dbutils.library.restartPython()

## Configuration

Set up authentication for different APIs:

In [ ]:
# Data Pipelines API client (existing)
pipelines_client = CustomerIOClient(
    api_key=os.getenv("CUSTOMERIO_API_KEY"),
    region="us"
)

# App API authentication (new)
app_auth = AppAPIAuth(
    api_token=os.getenv("CUSTOMERIO_APP_TOKEN"),
    region="us"
)

# Webhook secret (new)
webhook_secret = os.getenv("CUSTOMERIO_WEBHOOK_SECRET")

print("✓ All APIs configured")

## Data Pipelines API Usage

Use existing pipeline functions to manage customer data:

In [ ]:
# Identify a customer using Data Pipelines API
user_data = {
    "email": "test@example.com",
    "name": "Test User",
    "plan": "premium"
}

# This uses the existing utils
identify_result = identify_user(
    client=pipelines_client,
    user_id="user_123",
    traits=user_data
)

print(f"User identified: {identify_result}")

In [ ]:
# Track an event using Data Pipelines API
event_data = {
    "purchase_amount": 49.99,
    "product_id": "prod_456",
    "category": "software"
}

# This uses the existing utils
track_result = track_event(
    client=pipelines_client,
    user_id="user_123",
    event_name="purchase",
    properties=event_data
)

print(f"Event tracked: {track_result}")

## App API Usage

Use new App API functions for messaging:

In [ ]:
# Send transactional email using App API
transactional_result = send_transactional(
    auth=app_auth,
    transactional_message_id=123,
    to="test@example.com",
    message_data={
        "name": "Test User",
        "purchase_amount": "$49.99",
        "product_name": "Premium Software"
    }
)

print(f"Transactional email sent: {transactional_result}")

In [ ]:
# Send push notification using App API
push_result = send_push(
    auth=app_auth,
    identifiers={"id": "user_123"},
    title="Purchase Confirmed!",
    message="Thanks for your purchase of Premium Software",
    device_tokens=["device_token_789"]
)

print(f"Push notification sent: {push_result}")

## Webhook Processing

Process webhook events from Customer.IO:

In [ ]:
# Example webhook payload (would come from Customer.IO)
webhook_payload = json.dumps({
    "event_id": "evt_123abc",
    "event_type": "email_delivered",
    "timestamp": 1640995200,
    "data": {
        "customer_id": "user_123",
        "delivery_id": "del_456xyz",
        "subject": "Purchase Confirmation"
    }
})

# Example signature (would come from Customer.IO webhook header)
import hmac
import hashlib

signature_hash = hmac.new(
    webhook_secret.encode('utf-8'),
    webhook_payload.encode('utf-8'),
    hashlib.sha256
).hexdigest()

webhook_signature = f"sha256={signature_hash}"

print(f"Example webhook payload prepared")

In [ ]:
# Process webhook (verify, parse, identify)

# 1. Verify signature
is_valid = verify_signature(webhook_payload, webhook_signature, webhook_secret)
print(f"Webhook signature valid: {is_valid}")

if is_valid:
    # 2. Parse event
    event = parse_event(webhook_payload)
    print(f"Parsed event: {event['event_id']}")
    
    # 3. Get event type
    event_type = get_event_type(event)
    print(f"Event type: {event_type}")
    
    # 4. Process based on event type
    if event_type == "email_delivered":
        customer_id = event["data"]["customer_id"]
        print(f"Email delivered to customer: {customer_id}")
        
        # Could track this as an event in pipelines
        # track_event(pipelines_client, customer_id, "email_delivered", event["data"])
else:
    print("Invalid webhook signature - ignoring event")

## Multi-API Workflow Example

Complete workflow using all three APIs:

In [ ]:
def process_customer_purchase(user_id: str, purchase_data: dict):
    """
    Complete purchase workflow using multiple APIs.
    
    1. Track purchase event (Pipelines API)
    2. Send confirmation email (App API)
    3. Process delivery webhook when email sent (Webhook processing)
    """
    print(f"Processing purchase for user: {user_id}")
    
    # Step 1: Track purchase event
    track_result = track_event(
        client=pipelines_client,
        user_id=user_id,
        event_name="purchase_completed",
        properties=purchase_data
    )
    print(f"✓ Purchase tracked: {track_result}")
    
    # Step 2: Send confirmation email
    email_result = send_transactional(
        auth=app_auth,
        transactional_message_id=456,  # Purchase confirmation template
        identifiers={"id": user_id},
        message_data=purchase_data
    )
    print(f"✓ Confirmation email sent: {email_result}")
    
    # Step 3: Webhook processing would happen when Customer.IO delivers email
    print("✓ Workflow complete - webhook will be processed when email delivers")
    
    return {
        "purchase_tracked": track_result,
        "email_sent": email_result
    }

# Example usage
purchase_data = {
    "product_name": "Premium Software",
    "amount": 99.99,
    "currency": "USD"
}

workflow_result = process_customer_purchase("user_456", purchase_data)
print(f"\nWorkflow result: {workflow_result}")

## Summary

This notebook demonstrates:

1. **Data Pipelines API**: Customer identification and event tracking
2. **App API**: Transactional messaging and push notifications
3. **Webhook Processing**: Secure event processing and verification
4. **Multi-API Workflows**: Complete customer engagement workflows

Each API has its specific purpose:
- **Pipelines**: Data ingestion and customer profiles
- **App**: Message delivery and campaign management
- **Webhooks**: Event notifications and integrations

Together they provide comprehensive Customer.IO platform access.